# Q3. Import BERT for the classification model which is already fine-tuned and apply it to any text classification dataset such as the Twitter dataset. (5 points)

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.7 MB/s eta 0:00:00


# Import all the required libraries

In [ ]:
import requests
import json
import torch
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import BertTokenizerFast
from torch.utils.data import DataLoader, Dataset
from transformers import BertForQuestionAnswering
from transformers import AdamW
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class_df = pd.read_csv('/content/drive/MyDrive/train.tsv', delimiter='\t', encoding='ISO-8859-1')
class_df # reading the data

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [ ]:
class_df.Sentiment.unique()

array([1, 2, 3, 4, 0])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('textattack/bert-base-uncased-imdb')
from transformers import BertForSequenceClassification, BertConfig
config = BertConfig.from_pretrained('textattack/bert-base-uncased-imdb')
config.num_labels = 5
model = BertForSequenceClassification.from_pretrained('textattack/bert-base-uncased-imdb', config=config,ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at textattack/bert-base-uncased-imdb and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.classifier.weight.data.normal_(mean=0.0, std=config.initializer_range)
model.classifier.bias.data.zero_()

tensor([0., 0., 0., 0., 0.])

In [ ]:
import torch
encodings = tokenizer.batch_encode_plus(class_df['Phrase'].tolist(), truncation=True, padding=True, max_length=248, return_tensors='pt')
labels = class_df['Sentiment'].tolist()

In [ ]:
type(labels)

list

In [ ]:
from collections import Counter
counts = Counter(labels)
print(counts)

Counter({2: 79582, 3: 32927, 1: 27273, 4: 9206, 0: 7072})


In [ ]:
encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
type(encodings['input_ids'])

torch.Tensor

In [ ]:
label_ids = torch.tensor(labels) # Tensor Label
input_ids = encodings['input_ids']  # tensor Text
attention_mask = encodings['attention_mask']

In [ ]:
attention_mask.shape

torch.Size([156060, 80])

In [ ]:
input_ids.shape

torch.Size([156060, 80])

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')  # Device Checking
print(f'Working on {device}')

Working on cuda


In [ ]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
dataset = CustomDataset(encodings, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()
correct = 0
total = 0

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
all_labels = []
all_predictions = []
with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted = torch.max(logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_predictions)
accuracy = correct / total
print(f'Accuracy: {accuracy}')
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")

<ipython-input-26-3689e20369d3>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Accuracy: 0.21831346917852107
Precision: 0.18541350400342874
Recall: 0.21831346917852107
F1-score: 0.10556094536495826
Confusion Matrix:
[[  131     3  2885  4026    27]
 [  804    32  4881 21352   204]
 [ 3616   131  2211 72878   746]
 [  673    76   324 31558   296]
 [  118    19    33  8898   138]]


In [ ]:
#Data Accuracy without Finetuning
print(f'Accuracy for this data without finetuning BERT is {accuracy *100:.2f}%')

Accuracy for this data without finetuning BERT is 21.83%


#Q4. Fine-tune the BERT for the classification model on the text classification dataset you used in the previous question and check the performance. (10 points)

In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/train.tsv', delimiter='\t', encoding='ISO-8859-1')
data

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_and_encode(data, max_length=248):
    return tokenizer.batch_encode_plus(data, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
train_encodings = tokenize_and_encode(train_data['Phrase'].tolist())
test_encodings = tokenize_and_encode(test_data['Phrase'].tolist())


In [ ]:
max_len = 0
for Phrase in data['Phrase'].values:
    input_ids = tokenizer.encode(Phrase, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
print('Max sentence length: ', max_len)

Max sentence length:  80


In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
class MovieDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
train_dataset = MovieDataset(train_encodings, train_data['Sentiment'].tolist())
test_dataset = MovieDataset(test_encodings, test_data['Sentiment'].tolist())

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
train_loader

In [ ]:
N_EPOCHS = 6
optim = AdamW(model.parameters(), lr=5e-5)
model.to(device)
model.train()
for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    label = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=label)
    loss = outputs[0]
    loss.backward()
    optim.step()
    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/15606 [00:00<?, ?it/s]<ipython-input-36-aa168d9af629>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 6: 100%|██████████| 15606/15606 [33:23<00:00,  7.79it/s, loss=0.488]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm

In [ ]:
model.eval()
acc = []
all_labels = []
all_predictions = []
model.eval()
for batch in tqdm(valid_loader):
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())
accuracy = accuracy_score(all_labels, all_predictions)  # Metric Calculation
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_predictions)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")

  0%|          | 0/3902 [00:00<?, ?it/s]<ipython-input-36-aa168d9af629>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 3902/3902 [02:36<00:00, 24.98it/s]


Accuracy: 0.6795463283352556
Precision: 0.6770457694980494
Recall: 0.6795463283352556
F1-score: 0.6776426840134133
Confusion Matrix:
[[  758   581    73     3     1]
 [  680  3183  1592    69     3]
 [  149  1538 12503  1391    58]
 [   15   136  2065  3745   746]
 [    4     3    77   818  1021]]
